In [32]:
# ==========================================================================
# SECTION 1: Setup and Imports
# ==========================================================================
import sys
import os
import time
import warnings
import numpy as np
from dataclasses import dataclass
from typing import Optional, Dict, Any, List, Tuple
from collections import defaultdict
import traceback

warnings.filterwarnings('ignore')

# Ensure src is in path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Clear module cache for fresh import
import importlib
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]
print(f"Cleared {len(modules_to_clear)} cached modules")

# Import testing utilities
from qectostim.testing import load_all_decoders, STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL

# Import code discovery
from qectostim.codes import discover_all_codes

# Import noise models
from qectostim.noise.models import NoiseModel, CircuitDepolarizingNoise

# Import gadgets
from qectostim.gadgets import (
    # Transversal gates
    TransversalHadamard, TransversalS, TransversalT,
    TransversalX, TransversalY, TransversalZ,
    TransversalCNOT, TransversalCZ, TransversalSWAP,
    # Teleportation-based gates
    TeleportedHadamard, TeleportedS, TeleportedT, TeleportedIdentity,
    # CSS surgery
    LatticeZZMerge, LatticeXXMerge, SurgeryCNOT,
)

# Import experiment infrastructure
from qectostim.experiments.ft_gadget_experiment import FaultTolerantGadgetExperiment
from qectostim.gadgets.base import PhaseResult, PhaseType, ObservableTransform
from qectostim.gadgets.layout import QubitAllocation
from qectostim.experiments.stabilizer_rounds import DetectorContext
import stim

print("✓ All imports successful")

Cleared 100 cached modules
✓ All imports successful


In [34]:
# ==========================================================================
# Load decoders and codes
# ==========================================================================
decoder_classes = load_all_decoders()
print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

all_codes = discover_all_codes()
print(f"Discovered {len(all_codes)} codes")

# Standard noise model for testing
noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)
print(f"Noise model: CircuitDepolarizingNoise(p1=0.001, p2=0.001)")

# Select representative test codes (one per type, small)
from qectostim.codes.small import FourQubit422Code, PerfectCode513, SteanCode713
TEST_CODES = {
    'FourQubit422': FourQubit422Code(),
    'PerfectCode513': PerfectCode513(),
    'SteanCode713': SteanCode713(),
}
print(f"Test codes: {list(TEST_CODES.keys())}")

Loaded 11 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat']
Discovered 84 codes
Noise model: CircuitDepolarizingNoise(p1=0.001, p2=0.001)
Test codes: ['FourQubit422', 'PerfectCode513', 'SteanCode713']
Discovered 84 codes
Noise model: CircuitDepolarizingNoise(p1=0.001, p2=0.001)
Test codes: ['FourQubit422', 'PerfectCode513', 'SteanCode713']


## Section 2: Zero-Noise Verification

**Critical Sanity Check**: With no noise, the logical error rate MUST be 0.

If LER ≠ 0 with no noise, it indicates:
- Observable tracking bug (wrong qubits in OBSERVABLE_INCLUDE)
- Detector emission bug (wrong measurements referenced)
- Gate implementation bug (wrong physical operations)

In [35]:
# ==========================================================================
# SECTION 2: Zero-Noise Verification Helper
# ==========================================================================

def verify_zero_noise(
    gadget,
    codes: List,
    gadget_name: str,
    num_shots: int = 1000,
) -> Tuple[bool, float, Optional[str]]:
    """
    Verify that a gadget produces LER=0 with no noise.
    
    Returns (passed, ler, error_message)
    """
    try:
        exp = FaultTolerantGadgetExperiment(
            codes=codes,
            gadget=gadget,
            noise_model=None,  # NO NOISE!
            num_rounds_before=1,
            num_rounds_after=1,
            measurement_basis="Z",
        )
        
        passed, ler, error_msg = exp.verify_zero_noise_ler(num_shots)
        return passed, ler, error_msg
        
    except Exception as e:
        return False, 1.0, f"Exception: {e}"


def run_zero_noise_tests(gadgets_config: Dict, test_code, test_code_name: str):
    """
    Run zero-noise verification on all gadgets.
    """
    print("=" * 80)
    print(f"ZERO-NOISE VERIFICATION on {test_code_name}")
    print("=" * 80)
    print("Gadget                        | Status | LER    | Notes")
    print("-" * 80)
    
    results = {}
    for name, (cls, kwargs, is_two_qubit) in gadgets_config.items():
        try:
            gadget = cls(**kwargs)
            codes = [test_code, test_code] if is_two_qubit else [test_code]
            passed, ler, error_msg = verify_zero_noise(gadget, codes, name)
            
            status = "✓ PASS" if passed else "✗ FAIL"
            notes = error_msg if error_msg else ""
            print(f"{name:<30} | {status:<6} | {ler:.4f} | {notes[:35]}")
            results[name] = {'passed': passed, 'ler': ler, 'error': error_msg}
            
        except Exception as e:
            print(f"{name:<30} | ✗ ERR  | --     | {str(e)[:35]}")
            results[name] = {'passed': False, 'ler': None, 'error': str(e)}
    
    print("-" * 80)
    passed_count = sum(1 for r in results.values() if r['passed'])
    print(f"\nSummary: {passed_count}/{len(results)} passed zero-noise verification")
    
    return results

In [36]:
# ==========================================================================
# Define all gadgets for testing
# ==========================================================================

SINGLE_QUBIT_GADGETS = {
    'TransversalH': (TransversalHadamard, {'include_stabilizer_rounds': False}, False),
    'TransversalS': (TransversalS, {'include_stabilizer_rounds': False}, False),
    'TransversalT': (TransversalT, {'include_stabilizer_rounds': False}, False),
    'TransversalX': (TransversalX, {'include_stabilizer_rounds': False}, False),
    'TransversalY': (TransversalY, {'include_stabilizer_rounds': False}, False),
    'TransversalZ': (TransversalZ, {'include_stabilizer_rounds': False}, False),
}

TWO_QUBIT_GADGETS = {
    'TransversalCNOT': (TransversalCNOT, {'include_stabilizer_rounds': False}, True),
    'TransversalCZ': (TransversalCZ, {'include_stabilizer_rounds': False}, True),
    'TransversalSWAP': (TransversalSWAP, {'include_stabilizer_rounds': False}, True),
}

# Teleported gates take 1 code but internally create 2 blocks (data + ancilla)
# So is_two_qubit=False (we pass 1 code, not 2)
TELEPORTATION_GADGETS = {
    'TeleportedH': (TeleportedHadamard, {'include_stabilizer_rounds': False}, False),
    'TeleportedS': (TeleportedS, {'include_stabilizer_rounds': False}, False),
    'TeleportedT': (TeleportedT, {'include_stabilizer_rounds': False}, False),
    'TeleportedIdentity': (TeleportedIdentity, {'include_stabilizer_rounds': False}, False),
}

SURGERY_GADGETS = {
    'LatticeZZMerge': (LatticeZZMerge, {'num_merge_rounds': 1}, True),
    'LatticeXXMerge': (LatticeXXMerge, {'num_merge_rounds': 1}, True),
    'SurgeryCNOT': (SurgeryCNOT, {'num_rounds_before': 0, 'num_rounds_after': 0, 'num_merge_rounds': 1}, True),
}

ALL_GADGETS = {
    **SINGLE_QUBIT_GADGETS,
    **TWO_QUBIT_GADGETS,
    **TELEPORTATION_GADGETS,
    **SURGERY_GADGETS,
}

print(f"Defined {len(ALL_GADGETS)} gadgets for testing:")
print(f"  Single-qubit: {list(SINGLE_QUBIT_GADGETS.keys())}")
print(f"  Two-qubit: {list(TWO_QUBIT_GADGETS.keys())}")
print(f"  Teleportation (1 code, 2 blocks): {list(TELEPORTATION_GADGETS.keys())}")
print(f"  Surgery: {list(SURGERY_GADGETS.keys())}")

Defined 16 gadgets for testing:
  Single-qubit: ['TransversalH', 'TransversalS', 'TransversalT', 'TransversalX', 'TransversalY', 'TransversalZ']
  Two-qubit: ['TransversalCNOT', 'TransversalCZ', 'TransversalSWAP']
  Teleportation (1 code, 2 blocks): ['TeleportedH', 'TeleportedS', 'TeleportedT', 'TeleportedIdentity']
  Surgery: ['LatticeZZMerge', 'LatticeXXMerge', 'SurgeryCNOT']


In [37]:
# ==========================================================================
# Run Zero-Noise Verification on FourQubit422Code
# ==========================================================================
test_code = FourQubit422Code()
zero_noise_results = run_zero_noise_tests(ALL_GADGETS, test_code, 'FourQubit422Code')

ZERO-NOISE VERIFICATION on FourQubit422Code
Gadget                        | Status | LER    | Notes
--------------------------------------------------------------------------------
TransversalH                   | ✓ PASS | 0.0000 | 
TransversalS                   | ✓ PASS | 0.0000 | 
TransversalT                   | ✗ FAIL | 1.0000 | Circuit generation failed: Transver
TransversalX                   | ✓ PASS | 0.0000 | 
TransversalY                   | ✓ PASS | 0.0000 | 
TransversalZ                   | ✓ PASS | 0.0000 | 
TransversalCNOT                | ✓ PASS | 0.0000 | 
TransversalCZ                  | ✓ PASS | 0.0000 | 
TransversalSWAP                | ✓ PASS | 0.0000 | 
TeleportedH                    | ✓ PASS | 0.0000 | 
TeleportedS                    | ✗ FAIL | 0.4940 | Zero-noise LER = 0.4940 > 0.0 (obse
TeleportedT                    | ✗ FAIL | 0.4960 | Zero-noise LER = 0.4960 > 0.0 (obse
TeleportedIdentity             | ✗ FAIL | 0.4850 | Zero-noise LER = 0.4850 > 0.0 (obse
Tel

In [43]:
# Test the fix - rebuild experiment and check observable
from qectostim.gadgets.teleportation import TeleportedGate, TeleportationProtocol, AncillaState
from qectostim.codes.small.four_two_two_code import FourTwoTwoCode
import importlib
import qectostim.experiments.ft_gadget_experiment as exp_module
importlib.reload(exp_module)

# Create TeleportedIdentity
protocol = TeleportationProtocol("I", AncillaState.PLUS)
gadget = TeleportedGate(protocol)

code = FourTwoTwoCode()
exp = exp_module.FTGadgetExperiment(
    gadget=gadget,
    codes=[code],
    measurement_basis="Z",
    num_rounds_before=2,
    num_rounds_after=1,
)

circuit = exp.to_stim()
print("UPDATED CIRCUIT (looking for OBSERVABLE_INCLUDE):")
lines = str(circuit).split('\n')
for i, line in enumerate(lines):
    if 'OBSERVABLE_INCLUDE' in line or 'M ' in line:
        print(f"  Line {i}: {line}")

print("\n--- Full circuit ---")
print(circuit)

ModuleNotFoundError: No module named 'qectostim.codes.small.four_two_two_code'

## Section 3: Individual Gadget Tests

Test each gadget category with:
1. Observable transform verification
2. Circuit structure validation
3. Zero-noise LER check
4. Noisy LER check (should show error correction working)

In [16]:
# ==========================================================================
# Test Helper: Full Gadget Test
# ==========================================================================

def full_gadget_test(
    gadget_name: str,
    gadget_class,
    gadget_kwargs: Dict,
    test_code,
    is_two_qubit: bool = False,
    num_shots: int = 1000,
) -> Dict[str, Any]:
    """
    Run a full test suite on a single gadget.
    
    Returns dict with:
    - observable_transform: The transform applied by this gadget
    - circuit_valid: Whether circuit generates without error
    - zero_noise_ler: LER with no noise (should be 0)
    - noisy_ler: LER with noise
    - detectors: Number of detectors
    - observables: Number of observables
    """
    result = {
        'gadget_name': gadget_name,
        'is_two_qubit': is_two_qubit,
        'observable_transform': None,
        'circuit_valid': False,
        'zero_noise_ler': None,
        'zero_noise_passed': False,
        'noisy_ler': None,
        'detectors': 0,
        'observables': 0,
        'error': None,
    }
    
    try:
        gadget = gadget_class(**gadget_kwargs)
        codes = [test_code, test_code] if is_two_qubit else [test_code]
        
        # Get observable transform
        if hasattr(gadget, 'get_observable_transform'):
            try:
                obs_transform = gadget.get_observable_transform(test_code)
                result['observable_transform'] = str(obs_transform)
            except Exception as e:
                result['observable_transform'] = f"Error: {e}"
        
        # Test zero-noise circuit
        exp_zero = FaultTolerantGadgetExperiment(
            codes=codes,
            gadget=gadget_class(**gadget_kwargs),  # Fresh instance
            noise_model=None,
            num_rounds_before=1,
            num_rounds_after=1,
        )
        
        circuit = exp_zero.to_stim()
        result['circuit_valid'] = True
        result['detectors'] = circuit.num_detectors
        result['observables'] = circuit.num_observables
        
        # Zero-noise verification
        passed, ler, _ = exp_zero.verify_zero_noise_ler(num_shots)
        result['zero_noise_ler'] = ler
        result['zero_noise_passed'] = passed
        
        # Noisy LER
        noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)
        exp_noisy = FaultTolerantGadgetExperiment(
            codes=codes,
            gadget=gadget_class(**gadget_kwargs),  # Fresh instance
            noise_model=noise,
            num_rounds_before=2,
            num_rounds_after=2,
        )
        
        noisy_circuit = exp_noisy.to_stim()
        sampler = noisy_circuit.compile_detector_sampler()
        samples = sampler.sample(num_shots, append_observables=True)
        obs_col = samples[:, noisy_circuit.num_detectors]
        result['noisy_ler'] = float(np.mean(obs_col))
        
    except Exception as e:
        result['error'] = str(e)
        traceback.print_exc()
    
    return result


def print_gadget_test_result(result: Dict):
    """Pretty-print a gadget test result."""
    name = result['gadget_name']
    print(f"\n{'='*60}")
    print(f"Gadget: {name} ({'Two-Qubit' if result['is_two_qubit'] else 'Single-Qubit'})")
    print(f"{'='*60}")
    
    if result['error']:
        print(f"  ✗ ERROR: {result['error'][:60]}")
        return
    
    print(f"  Observable Transform: {result['observable_transform']}")
    print(f"  Circuit Valid: {'✓' if result['circuit_valid'] else '✗'}")
    print(f"  Detectors: {result['detectors']}")
    print(f"  Observables: {result['observables']}")
    
    zn_status = '✓ PASS' if result['zero_noise_passed'] else '✗ FAIL'
    zn_ler = result['zero_noise_ler']
    print(f"  Zero-Noise LER: {zn_ler:.4f} ({zn_status})")
    
    if result['noisy_ler'] is not None:
        print(f"  Noisy LER (p=0.001): {result['noisy_ler']:.4f}")

In [9]:
# ==========================================================================
# TEST: Single-Qubit Transversal Gates
# ==========================================================================
print("\n" + "#" * 80)
print("# SINGLE-QUBIT TRANSVERSAL GATES")
print("#" * 80)

test_code = FourQubit422Code()
single_qubit_results = {}

for name, (cls, kwargs, is_two_qubit) in SINGLE_QUBIT_GADGETS.items():
    result = full_gadget_test(name, cls, kwargs, test_code, is_two_qubit)
    single_qubit_results[name] = result
    print_gadget_test_result(result)


################################################################################
# SINGLE-QUBIT TRANSVERSAL GATES
################################################################################

Gadget: TransversalH (Single-Qubit)
  Observable Transform: Error: TransversalHadamard.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 4
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0140

Gadget: TransversalS (Single-Qubit)
  Observable Transform: Error: TransversalS.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 4
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0080

Gadget: TransversalT (Single-Qubit)
  ✗ ERROR: Gate not found: 'T'


Traceback (most recent call last):
  File "/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5641/3569325069.py", line 58, in full_gadget_test
    circuit = exp_zero.to_stim()
              ^^^^^^^^^^^^^^^^^^
  File "/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/experiments/ft_gadget_experiment.py", line 574, in to_stim
    result = self.gadget.emit_next_phase(circuit, unified_alloc, ctx)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/gadgets/transversal.py", line 243, in emit_next_phase
    circuit.append(self.stim_gate, qubits)
IndexError: Gate not found: 'T'



Gadget: TransversalX (Single-Qubit)
  Observable Transform: Error: TransversalX.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 4
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0050

Gadget: TransversalY (Single-Qubit)
  Observable Transform: Error: TransversalY.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 4
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0100

Gadget: TransversalZ (Single-Qubit)
  Observable Transform: Error: TransversalZ.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 4
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0080


In [17]:
# ==========================================================================
# TEST: Two-Qubit Transversal Gates
# ==========================================================================
print("\n" + "#" * 80)
print("# TWO-QUBIT TRANSVERSAL GATES")
print("#" * 80)

test_code = FourQubit422Code()
two_qubit_results = {}

for name, (cls, kwargs, is_two_qubit) in TWO_QUBIT_GADGETS.items():
    result = full_gadget_test(name, cls, kwargs, test_code, is_two_qubit)
    two_qubit_results[name] = result
    print_gadget_test_result(result)


################################################################################
# TWO-QUBIT TRANSVERSAL GATES
################################################################################

Gadget: TransversalCNOT (Two-Qubit)
  Observable Transform: Error: TransversalCNOT.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 8
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0160

Gadget: TransversalCZ (Two-Qubit)
  Observable Transform: Error: TransversalCZ.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 8
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.0110

Gadget: TransversalSWAP (Two-Qubit)
  Observable Transform: Error: TransversalSWAP.get_observable_transform() takes 1 positional argument but 2 were given
  Circuit Valid: ✓
  Detectors: 8
  Observables: 1
  Zero-Noise LER: 0.0000 (✓ PASS)
  Noisy LER (p=0.001): 0.015

In [ ]:
# ==========================================================================
# TEST: Teleportation-Based Gates
# ==========================================================================
print("\n" + "#" * 80)
print("# TELEPORTATION-BASED GATES")
print("#" * 80)

test_code = FourQubit422Code()
teleportation_results = {}

for name, (cls, kwargs, is_two_qubit) in TELEPORTATION_GADGETS.items():
    result = full_gadget_test(name, cls, kwargs, test_code, is_two_qubit)
    teleportation_results[name] = result
    print_gadget_test_result(result)

In [ ]:
# ==========================================================================
# TEST: Lattice Surgery Operations
# ==========================================================================
print("\n" + "#" * 80)
print("# LATTICE SURGERY OPERATIONS")
print("#" * 80)

test_code = FourQubit422Code()
surgery_results = {}

for name, (cls, kwargs, is_two_qubit) in SURGERY_GADGETS.items():
    result = full_gadget_test(name, cls, kwargs, test_code, is_two_qubit)
    surgery_results[name] = result
    print_gadget_test_result(result)

## Section 4: Decoder Compatibility Matrix

Test each gadget category with all available decoders across multiple codes.

In [ ]:
# ==========================================================================
# Decoder Compatibility Test Infrastructure
# ==========================================================================

STATUS_OK = 'OK'
STATUS_WARN = 'WARN'
STATUS_FAIL = 'FAIL'
STATUS_SKIP = 'SKIP'
STATUS_NA = 'N/A'

def test_gadget_decoder(
    code,
    code_name: str,
    gadget_class,
    gadget_kwargs: Dict,
    decoder_class,
    decoder_name: str,
    is_two_qubit: bool = False,
    shots: int = 1000,
) -> Dict[str, Any]:
    """
    Test a gadget+decoder combination on a code.
    """
    result = {
        'status': STATUS_FAIL,
        'ler': None,
        'error': None,
    }
    
    try:
        gadget = gadget_class(**gadget_kwargs)
        codes = [code, code] if is_two_qubit else [code]
        
        noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)
        exp = FaultTolerantGadgetExperiment(
            codes=codes,
            gadget=gadget,
            noise_model=noise,
            num_rounds_before=2,
            num_rounds_after=2,
        )
        
        circuit = exp.to_stim()
        
        if circuit.num_detectors == 0:
            result['status'] = STATUS_SKIP
            result['error'] = 'No detectors'
            return result
        
        # Try to decode
        try:
            dem = circuit.detector_error_model(decompose_errors=True)
            decoder = decoder_class.from_detector_error_model(dem)
            
            sampler = circuit.compile_detector_sampler()
            samples = sampler.sample(shots, append_observables=True)
            
            det_samples = samples[:, :circuit.num_detectors]
            obs_samples = samples[:, circuit.num_detectors:]
            
            predictions = decoder.decode_batch(det_samples)
            if len(predictions.shape) == 1:
                predictions = predictions.reshape(-1, 1)
            
            errors = np.any(predictions != obs_samples, axis=1)
            result['ler'] = float(np.mean(errors))
            result['status'] = STATUS_OK
            
        except Exception as e:
            result['status'] = STATUS_WARN
            result['error'] = str(e)[:50]
    
    except Exception as e:
        result['status'] = STATUS_FAIL
        result['error'] = str(e)[:50]
    
    return result


def run_decoder_matrix(
    gadgets_config: Dict,
    test_codes: Dict,
    decoder_classes: Dict,
    category_name: str,
) -> Dict:
    """
    Run decoder compatibility matrix for a gadget category.
    """
    print("=" * 140)
    print(f"DECODER COMPATIBILITY: {category_name}")
    print("=" * 140)
    
    dec_names = list(decoder_classes.keys())
    
    header = f"{'Gadget':<20} | {'Code':<20}"
    for dec in dec_names:
        header += f" | {dec[:8]:^8}"
    print(header)
    print("-" * len(header))
    
    all_results = {}
    
    for gadget_name, (gadget_class, gadget_kwargs, is_two_qubit) in gadgets_config.items():
        all_results[gadget_name] = {}
        
        for code_name, code in test_codes.items():
            all_results[gadget_name][code_name] = {}
            
            row = f"{gadget_name:<20} | {code_name:<20}"
            
            for dec_name in dec_names:
                result = test_gadget_decoder(
                    code=code,
                    code_name=code_name,
                    gadget_class=gadget_class,
                    gadget_kwargs=gadget_kwargs,
                    decoder_class=decoder_classes[dec_name],
                    decoder_name=dec_name,
                    is_two_qubit=is_two_qubit,
                )
                
                all_results[gadget_name][code_name][dec_name] = result
                
                if result['status'] == STATUS_OK:
                    cell = f"{result['ler']:.3f}" if result['ler'] is not None else "  OK  "
                elif result['status'] == STATUS_SKIP:
                    cell = " SKIP "
                elif result['status'] == STATUS_WARN:
                    cell = " WARN "
                else:
                    cell = " FAIL "
                
                row += f" | {cell:^8}"
            
            print(row)
    
    print("-" * len(header))
    return all_results

In [ ]:
# ==========================================================================
# DECODER MATRIX: Single-Qubit Gates
# ==========================================================================
single_qubit_decoder_results = run_decoder_matrix(
    SINGLE_QUBIT_GADGETS,
    TEST_CODES,
    decoder_classes,
    "Single-Qubit Transversal Gates",
)

In [ ]:
# ==========================================================================
# DECODER MATRIX: Two-Qubit Gates
# ==========================================================================
two_qubit_decoder_results = run_decoder_matrix(
    TWO_QUBIT_GADGETS,
    TEST_CODES,
    decoder_classes,
    "Two-Qubit Transversal Gates",
)

In [ ]:
# ==========================================================================
# DECODER MATRIX: Teleportation Gates
# ==========================================================================
teleportation_decoder_results = run_decoder_matrix(
    TELEPORTATION_GADGETS,
    TEST_CODES,
    decoder_classes,
    "Teleportation-Based Gates",
)

In [ ]:
# ==========================================================================
# DECODER MATRIX: Surgery Operations
# ==========================================================================
surgery_decoder_results = run_decoder_matrix(
    SURGERY_GADGETS,
    TEST_CODES,
    decoder_classes,
    "Lattice Surgery Operations",
)

## Section 5: Summary

In [ ]:
# ==========================================================================
# FINAL SUMMARY
# ==========================================================================

print("=" * 80)
print("COMPREHENSIVE GADGET TEST SUMMARY")
print("=" * 80)

# Zero-noise results
print("\n--- Zero-Noise Verification ---")
zn_passed = sum(1 for r in zero_noise_results.values() if r['passed'])
print(f"  Passed: {zn_passed}/{len(zero_noise_results)}")

if zn_passed < len(zero_noise_results):
    print("  Failed gadgets:")
    for name, r in zero_noise_results.items():
        if not r['passed']:
            print(f"    - {name}: LER={r['ler']:.4f}")

# Individual gadget results
print("\n--- Individual Gadget Tests ---")
all_individual = {
    **single_qubit_results,
    **two_qubit_results,
    **teleportation_results,
    **surgery_results,
}

for category, results in [
    ('Single-Qubit', single_qubit_results),
    ('Two-Qubit', two_qubit_results),
    ('Teleportation', teleportation_results),
    ('Surgery', surgery_results),
]:
    valid = sum(1 for r in results.values() if r['circuit_valid'])
    zn_ok = sum(1 for r in results.values() if r['zero_noise_passed'])
    print(f"  {category}: {valid}/{len(results)} circuits valid, {zn_ok}/{len(results)} zero-noise OK")

print("\n" + "=" * 80)
print("END OF COMPREHENSIVE GADGET TEST")
print("=" * 80)